In [1]:
import pandas as pd

In [2]:
sales = pd.read_csv('../data/redfin_sales.tsv000', sep= '\t')
values = pd.read_csv('../data/zhvi_values.csv')

In [3]:
sales.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2024-09-01,2024-09-30,30,metro,-2,42740,f,"Sedalia, MO metro area",NaN,NaN,...,-0.032143,0.320388,0.059519,0.088681,0.372549,0.039216,-0.098039,"Sedalia, MO",42740,2025-01-13 14:16:48
1,2020-07-01,2020-07-31,30,metro,-2,15140,f,"Brownsville, TN metro area",NaN,NaN,...,-0.285714,NaN,NaN,NaN,0.000000,-0.142857,0.000000,"Brownsville, TN",15140,2025-01-13 14:16:48
2,2022-07-01,2022-07-31,30,metro,-2,21660,f,"Eugene, OR metro area",NaN,NaN,...,-0.116667,0.172414,-0.027586,-0.068966,0.500000,-0.038462,-0.041667,"Eugene, OR",21660,2025-01-13 14:16:48
3,2024-09-01,2024-09-30,30,metro,-2,43940,f,"Spearfish, SD metro area",NaN,NaN,...,-0.083843,0.201493,-0.033582,0.054600,0.295455,-0.079545,0.103965,"Spearfish, SD",43940,2025-01-13 14:16:48
4,2021-02-01,2021-02-28,30,metro,-2,19860,f,"Dickinson, ND metro area",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Dickinson, ND",19860,2025-01-13 14:16:48


In [4]:
values.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
0,102001,0,United States,country,NaN,120362.209156,120573.964867,120836.573128,121399.816300,122048.041664,...,350187.830292,352016.063401,353428.262523,353981.424700,354127.238007,354366.950053,354810.776247,355390.466837,355876.097911,356585.270651
1,394913,1,"New York, NY",msa,NY,215827.415564,216744.560306,217670.199462,219545.902568,221488.258090,...,644520.011902,650611.568592,655257.357090,658329.448382,661068.322561,664800.522200,668815.751121,672329.954298,675092.682472,676871.828990
2,753899,2,"Los Angeles, CA",msa,CA,218392.422126,219205.119030,220287.623555,222441.443922,224797.048177,...,907196.864855,910626.100548,916455.723307,920367.443074,924697.980572,929829.637744,936186.740152,941354.595913,945807.243380,949787.229824
3,394463,3,"Chicago, IL",msa,IL,150388.567381,150527.783385,150792.922716,151452.523481,152245.943863,...,309063.307733,311880.938371,313961.458492,315161.920489,315884.255989,316789.130036,317773.867511,318733.494209,319752.751523,320977.001399
4,394514,4,"Dallas, TX",msa,TX,125052.500914,125108.199735,125172.231232,125338.514302,125557.753977,...,370636.213403,371597.053072,371739.693490,371080.749556,370149.833607,369503.923660,369307.775226,369202.798720,368916.953457,368397.480322


Removing columns that will not be used, filtering just for TN, and for property types of all residential. 

In [5]:
sales = sales.loc[(sales['property_type_id'] == -1) & (sales['state_code'] == 'TN'),  
    ['period_end', 'state_code', 'parent_metro_region', 'property_type', 'median_sale_price', 'median_list_price', 'median_ppsf', 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'price_drops', 'off_market_in_two_weeks']]
    

In [8]:
values = (
    values.loc[values['StateName'] == 'TN']
    .drop(columns=['RegionID', 'SizeRank', 'RegionType'])
    )

In [9]:
values = values.melt(
    id_vars = ['RegionName', 'StateName'], 
    var_name = 'period_end',
    value_name = 'estimated_value')

Merging dataframes, dropped rows that were not in both tables, and dropped duplicate columns. 
- With the merge the were 26 metro regions in both dataframes except each had one that did not match in the other. (Chattanooga and Brownsville)
- For now keeping it to the min date that was in both tables for the region, currently not filtering for the minimum date for where there was data. 

In [18]:
merged = (pd.merge(
    left = sales,
    right = values,
    left_on = ['parent_metro_region', 'period_end'], 
    right_on = ['RegionName', 'period_end'],
    how = 'inner',
    validate = 'one_to_one'
    ).drop(columns = ['RegionName', 'StateName'])
)

In [19]:
merged.groupby('parent_metro_region')['period_end'].min()

parent_metro_region
Athens, TN          2012-01-31
Clarksville, TN     2012-07-31
Cleveland, TN       2012-01-31
Cookeville, TN      2012-01-31
Crossville, TN      2012-01-31
Dayton, TN          2012-01-31
Dyersburg, TN       2012-02-29
Greeneville, TN     2012-01-31
Jackson, TN         2012-04-30
Johnson City, TN    2012-02-29
Kingsport, TN       2012-01-31
Knoxville, TN       2012-01-31
Lawrenceburg, TN    2012-01-31
Lewisburg, TN       2012-01-31
Martin, TN          2013-08-31
McMinnville, TN     2012-01-31
Memphis, TN         2012-01-31
Morristown, TN      2012-01-31
Nashville, TN       2013-02-28
Newport, TN         2012-01-31
Paris, TN           2012-02-29
Sevierville, TN     2012-01-31
Shelbyville, TN     2012-01-31
Tullahoma, TN       2012-01-31
Union City, TN      2013-09-30
Name: period_end, dtype: object

Saving merged dataframe to csv

In [20]:
merged.to_csv('../data/TN_sales.csv', index = False)